<center><h1>Moore_Andrew_HW3</h1></center>
<br>
<br>

Name: Andrew Moore
<br>
Github Username: drewm8080
<br>
USC ID: 7583780941

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [4]:
import pandas as pd 
import numpy as np

from scipy.stats import bootstrap


Get the AReM Data Set

In [5]:
folders= ['bending1', 'bending2','cycling','lying','sitting','standing','walking']
train = []
test = []
bending1_data_range= range(1,8)
bending2_data_range = range(1,7)
other_data_range = range(1,16)
for file in folders:
    # handling bending data
    if file == 'bending1': 
        for i in bending1_data_range:
            # reading in the data 
            df= pd.read_csv(f'../data/{file}/dataset{i}.csv',skiprows= 4)
            if (i == 1) or (i==2):
                test.append(df)
            else:
                train.append(df)
    # bending data 2 has a different amount of instances
    elif file== 'bending2':
             # reading in the data 
            for i in bending2_data_range:
                df= pd.read_csv(f'../data/{file}/dataset{i}.csv',skiprows= 4)
                if (i == 1) or (i==2):
                    test.append(df)
                else:
                    train.append(df)
        
        
    # handling all other types of data             
    else:
        for i in other_data_range:
            df= pd.read_csv(f'../data/{file}/dataset{i}.csv',skiprows= 4)
            df=df.dropna()
            if (i == 1) or (i==2) or (i==3):
                test.append(df)
            else:
                train.append(df)

# training                 
train_df = pd.concat(train,axis=0)
train_df = train_df.drop(columns = '# Columns: time')


# testing 
test_df = pd.concat(test,axis=0)
test_df = test_df.drop(columns = '# Columns: time')

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33119 entries, 0 to 479
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   avg_rss12  33119 non-null  float64
 1   var_rss12  33119 non-null  float64
 2   avg_rss13  33119 non-null  float64
 3   var_rss13  33119 non-null  float64
 4   avg_rss23  33119 non-null  float64
 5   var_rss23  33119 non-null  float64
dtypes: float64(6)
memory usage: 1.8 MB


In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9120 entries, 0 to 479
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   avg_rss12  9120 non-null   float64
 1   var_rss12  9120 non-null   float64
 2   avg_rss13  9120 non-null   float64
 3   var_rss13  9120 non-null   float64
 4   avg_rss23  9120 non-null   float64
 5   var_rss23  9120 non-null   float64
dtypes: float64(6)
memory usage: 498.8 KB


In [7]:
df = pd.DataFrame(columns = ["min1", "max1", "mean1","median1",'std dev1','1st Q1','3rd Q1',
                            "min2", "max2", "mean2","median2",'std dev2','1st Q2','3rd Q2',
                            "min3", "max3", "mean3","median3",'std dev3','1st Q3','3rd Q3',
                            "min4", "max4", "mean4","median4",'std dev4','1st Q4','3rd Q4',
                            "min5", "max5", "mean5","median5",'std dev5','1st Q5','3rd Q5',
                            "min6", "max6", "mean6","median6",'std dev6','1st Q6','3rd Q6'])
print(df)

Empty DataFrame
Columns: [min1, max1, mean1, median1, std dev1, 1st Q1, 3rd Q1, min2, max2, mean2, median2, std dev2, 1st Q2, 3rd Q2, min3, max3, mean3, median3, std dev3, 1st Q3, 3rd Q3, min4, max4, mean4, median4, std dev4, 1st Q4, 3rd Q4, min5, max5, mean5, median5, std dev5, 1st Q5, 3rd Q5, min6, max6, mean6, median6, std dev6, 1st Q6, 3rd Q6]
Index: []

[0 rows x 42 columns]


### (c) Feature Extraction

#### i. Research

reference -- https://stats.stackexchange.com/questions/50807/features-for-time-series-classification
reference --https://www.researchgate.net/figure/List-of-extracted-time-domain-features-from-each-window-size_tbl2_327523090#:~:text=extracted%20time%2Ddomain%20features%20are,Root%20Mean%20Square%2C%20and%20Distance.


The time-domain features that are most important are the mean, std deviation, min and max values, variance, covariance, and zero cross rate

In [8]:
test = test_df.describe().transpose()
test

,count,mean,std,min,25%,50%,75%,max
avg_rss12,33119.0,39.005285,5.986658,0.0,35.25,39.67,43.50,56.25
var_rss12,33119.0,1.510208,2.082433,0.0,0.43,0.50,1.92,17.24
avg_rss13,33119.0,14.166349,5.346573,0.0,11.50,14.75,18.00,32.75
var_rss13,33119.0,1.505252,1.648251,0.0,0.43,0.83,2.29,11.42
avg_rss23,33119.0,15.669678,6.287338,0.0,12.00,15.67,18.75,40.33
var_rss23,33119.0,1.644033,1.646942,0.0,0.47,1.00,2.38,13.61


#### ii. Extraction

In [9]:
def statistics(dataframe):
    cleaned_df = dataframe.drop(columns='# Columns: time')
    df_stats = cleaned_df.describe().transpose()
    # getting stats
    min1 = df_stats.iloc[0,3]
    min2 = df_stats.iloc[1,3]
    min3 = df_stats.iloc[2,3]
    min4 = df_stats.iloc[3,3]
    min5 = df_stats.iloc[4,3]
    min6 = df_stats.iloc[5,3]
    
    max1= df_stats.iloc[0,-1]
    max2= df_stats.iloc[1,-1]
    max3= df_stats.iloc[2,-1]
    max4= df_stats.iloc[3,-1]
    max5= df_stats.iloc[4,-1]
    max6= df_stats.iloc[5,-1]
    
    mean1= df_stats.iloc[0,1]
    mean2= df_stats.iloc[1,1]
    mean3= df_stats.iloc[2,1]
    mean4= df_stats.iloc[3,1]
    mean5= df_stats.iloc[4,1]
    mean6= df_stats.iloc[5,1]
    
    std1= df_stats.iloc[0,2]
    std2= df_stats.iloc[1,2]
    std3= df_stats.iloc[2,2]
    std4= df_stats.iloc[3,2]
    std5= df_stats.iloc[4,2]
    std6= df_stats.iloc[5,2]
    
    q11= df_stats.iloc[0,4]
    q12= df_stats.iloc[1,4]
    q13= df_stats.iloc[2,4]
    q14= df_stats.iloc[3,4]
    q15= df_stats.iloc[4,4]
    q16= df_stats.iloc[5,4]
    
    q31= df_stats.iloc[0,6]
    q32= df_stats.iloc[1,6]
    q33= df_stats.iloc[2,6]
    q34= df_stats.iloc[3,6]
    q35= df_stats.iloc[4,6]
    q36= df_stats.iloc[5,6]
    
    median1= df_stats.iloc[0,5]
    median2= df_stats.iloc[1,5]
    median3= df_stats.iloc[2,5]
    median4= df_stats.iloc[3,5]
    median5= df_stats.iloc[4,5]
    median6= df_stats.iloc[5,5]
    
    final_list = [min1,max1,mean1,median1,std1,q11,q31,
                 min2,max2,mean2,median2,std2,q12,q32,
                 min3,max3,mean3,median3,std3,q13,q33,
                 min4,max4,mean4,median4,std4,q14,q34,
                 min5,max5,mean5,median5,std5,q15,q35,
                 min6,max6,mean6,median6,std6,q16,q36]
    
    return final_list


In [10]:
folders= ['bending1', 'bending2','cycling','lying','sitting','standing','walking']
train = []
test = []
bending1_data_range= range(1,8)
bending2_data_range = range(1,7)
other_data_range = range(1,16)


time_domain_features = pd.DataFrame(columns = ["min1", "max1", "mean1","median1",'std dev1','1st Q1','3rd Q1',
                            "min2", "max2", "mean2","median2",'std dev2','1st Q2','3rd Q2',
                            "min3", "max3", "mean3","median3",'std dev3','1st Q3','3rd Q3',
                            "min4", "max4", "mean4","median4",'std dev4','1st Q4','3rd Q4',
                            "min5", "max5", "mean5","median5",'std dev5','1st Q5','3rd Q5',
                            "min6", "max6", "mean6","median6",'std dev6','1st Q6','3rd Q6'])
stat_list = []
for file in folders:
    # handling bending data
    if file == 'bending1': 
        for i in bending1_data_range:
            # reading in the data 
            df= pd.read_csv(f'../data/{file}/dataset{i}.csv',skiprows= 4)
            if (i == 1) or (i==2):
                result = statistics(df)
                stat_list.append(result)
            else:
                result = statistics(df)
                stat_list.append(result)
    # bending data 2 has a different amount of instances
    elif file== 'bending2':
             # reading in the data 
            for i in bending2_data_range:
                df= pd.read_csv(f'../data/{file}/dataset{i}.csv',skiprows= 4)
                if (i == 1) or (i==2):
                    result = statistics(df)
                    stat_list.append(result)
                else:
                    result = statistics(df)
                    stat_list.append(result)
        
    # handling all other types of data             
    else:
        for i in other_data_range:
            df= pd.read_csv(f'../data/{file}/dataset{i}.csv',skiprows= 4)
            if (i == 1) or (i==2) or (i==3):
                result = statistics(df)
                stat_list.append(result)
            else:
                result = statistics(df)
                stat_list.append(result)


In [11]:
full_stats_df = pd.DataFrame(stat_list,columns= time_domain_features.columns)
full_stats_df

,min1,max1,mean1,median1,std dev1,1st Q1,3rd Q1,min2,max2,mean2,...,std dev5,1st Q5,3rd Q5,min6,max6,mean6,median6,std dev6,1st Q6,3rd Q6
0,37.25,45.00,40.624792,40.50,1.476967,39.25,42.0000,0.0,1.30,0.358604,...,2.188449,33.0000,36.00,0.00,1.92,0.570583,0.43,0.582915,0.0000,1.3000
1,38.00,45.67,42.812812,42.50,1.435550,42.00,43.6700,0.0,1.22,0.372438,...,1.995255,32.0000,34.50,0.00,3.11,0.571083,0.43,0.601010,0.0000,1.3000
2,35.00,47.40,43.954500,44.33,1.558835,43.00,45.0000,0.0,1.70,0.426250,...,1.999604,35.3625,36.50,0.00,1.79,0.493292,0.43,0.513506,0.0000,0.9400
3,33.00,47.75,42.179813,43.50,3.670666,39.15,45.0000,0.0,3.00,0.696042,...,3.849448,30.4575,36.33,0.00,2.18,0.613521,0.50,0.524317,0.0000,1.0000
4,33.00,45.75,41.678063,41.75,2.243490,41.33,42.7500,0.0,2.83,0.535979,...,2.411026,28.4575,31.25,0.00,1.79,0.383292,0.43,0.389164,0.0000,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,19.50,45.33,33.586875,34.25,4.650935,30.25,37.0000,0.0,14.67,4.576562,...,3.283983,13.7300,18.25,0.00,8.32,3.259729,3.11,1.640243,2.0500,4.3225
84,19.75,45.50,34.322750,35.25,4.752477,31.00,38.0000,0.0,13.47,4.456333,...,3.119856,13.5000,17.75,0.00,9.67,3.432562,3.20,1.732727,2.1575,4.5650
85,19.50,46.00,34.546229,35.25,4.842294,31.25,37.8125,0.0,12.47,4.371958,...,2.823124,14.0000,17.75,0.00,10.00,3.338125,3.08,1.656742,2.1600,4.3350
86,23.50,46.25,34.873229,35.25,4.531720,31.75,38.2500,0.0,14.82,4.380583,...,3.131076,13.7500,18.00,0.00,9.51,3.424646,3.27,1.690960,2.1700,4.5000


#### iii. Standard Deviation

In [99]:
full_stats_df.describe().loc[['std']].transpose()

,std
min1,9.569975
max1,4.394362
mean1,5.335718
median1,5.440054
std dev1,1.772153
1st Q1,6.153590
3rd Q1,5.138925
min2,0.000000
max2,5.062729
mean2,1.574164


In [21]:
# reference -- https://www.statology.org/bootstrapping-in-python/
columns = full_stats_df.columns

for column in columns:
    data = (full_stats_df[f'{column}'],)
    bootstrap_ci = bootstrap(data, np.median, confidence_level=0.90,
                             random_state=1, method='percentile')
    print("The confidence interval for",column,"is",bootstrap_ci.confidence_interval)
    

The confidence interval for min1 is ConfidenceInterval(low=26.75, high=33.25)
The confidence interval for max1 is ConfidenceInterval(low=45.25, high=46.25)
The confidence interval for mean1 is ConfidenceInterval(low=37.1143125, high=42.11158333333333)
The confidence interval for median1 is ConfidenceInterval(low=36.33, high=41.5)
The confidence interval for std dev1 is ConfidenceInterval(low=2.384169571190206, high=3.555786994948472)
The confidence interval for 1st Q1 is ConfidenceInterval(low=34.5, high=39.75)
The confidence interval for 3rd Q1 is ConfidenceInterval(low=39.75, high=42.875)
The confidence interval for min2 is ConfidenceInterval(low=0.0, high=0.0)
The confidence interval for max2 is ConfidenceInterval(low=3.0, high=6.4)
The confidence interval for mean2 is ConfidenceInterval(low=0.440687500000001, high=0.5375729166666674)
The confidence interval for median2 is ConfidenceInterval(low=0.47, high=0.5)
The confidence interval for std dev2 is ConfidenceInterval(low=0.3938656

#### iv. Select Features

I would choose min,max and mean. Min and max would be the best for detecting outliers and mean would be the best for understanding how the average value is changing over time.

## 2. ISLR 3.7.4

### (a) Linear Train

I would exect the fit from the polynomial regression to have a lower training RSS because it would overfit the data. 

### (b) Linear Test

I would expect the polynomial regression to have higher test RSS because the overfit, it wont be able to capture the true nature of the data

### (c) Not Linear Train

I would expect the model that can fit the flexible data better (polynomial) to have a lower training RSS and would capture the data more accuratly.

### (d) Not Linear Testing

Given that we don't know how far from linear the data is, we cannot say which model would be better. If the relationship is not linear, the polynomial model would do better. If the relationship is linear, the linear model would do better. 

## 3. ISLR 3.7.3 - Extra Practice 

## 3. ISLR 3.7.5 - Extra Practice 